In [3]:
import os
import cv2
import numpy as np
import pickle
from tqdm import tqdm
from typing import List

from sklearn.cluster import MiniBatchKMeans, KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score
from scipy.stats import multivariate_normal
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold

import optuna
from optuna.integration.wandb import WeightsAndBiasesCallback
import wandb

# optuna and w&b setup

In [4]:
# wandb_kwargs = {"project": "FINDING_BEST_COMBINATION_GUNJAN"}
# wandbc = WeightsAndBiasesCallback(wandb_kwargs=wandb_kwargs, as_multirun=True)

# data directory

In [5]:
train_images_filenames = pickle.load(
    open("./MIT_split/train_images_filenames.dat", "rb")
)
test_images_filenames = pickle.load(open("./MIT_split/test_images_filenames.dat", "rb"))
train_images_filenames = ["." + n[15:] for n in train_images_filenames]
test_images_filenames = ["." + n[15:] for n in test_images_filenames]
train_labels = pickle.load(open("./MIT_split/train_labels.dat", "rb"))
test_labels = pickle.load(open("./MIT_split/test_labels.dat", "rb"))



# local feature extractor

In [6]:
def SIFT_des(img: np.array, pixel_interval: int = None):
    sift = cv2.SIFT_create()

    # dense representation based on pixel interval
    if pixel_interval is not None:
        keypoints = [
            cv2.KeyPoint(x, y, pixel_interval)
            for y in range(0, img.shape[0], pixel_interval)
            for x in range(0, img.shape[1], pixel_interval)
        ]
        _, descriptors = sift.compute(img, keypoints)
    # regular keypoint representation
    else:
        keypoints = sift.detect(img, None)
        _, descriptors = sift.compute(img, keypoints)
    return descriptors


def ORB_des(img, pixel_interval: int = None):
    orb = cv2.ORB_create()
    if pixel_interval is not None:
        keypoints = [
            cv2.KeyPoint(x, y, pixel_interval)
            for y in range(0, img.shape[0], pixel_interval)
            for x in range(0, img.shape[1], pixel_interval)
        ]
        _, descriptors = orb.compute(img, keypoints)
    else:
        keypoints = orb.detect(img, None)
        _, descriptors = orb.compute(img, keypoints)
    return descriptors


def FAST_des(img, pixel_interval: int = None):
    fast = cv2.FastFeatureDetector_create()

    if pixel_interval is not None:
        keypoints = [
            cv2.KeyPoint(x, y, pixel_interval)
            for y in range(0, img.shape[0], pixel_interval)
            for x in range(0, img.shape[1], pixel_interval)
        ]
    else:
        keypoints = fast.detect(img, None)

    # Compute descriptors using ORB
    orb = cv2.ORB_create()
    _, descriptors = orb.compute(img, keypoints)

    return descriptors


def get_descriptor_feature(
    img: np.array, descriptor_name: str, pixel_interval: int = None
):
    """
    Wrapper function: Compute and return the descriptors for the given image using the specified feature detection algorithm.

    Args:
        img (np.array): input image 

        descriptor_name (str): Name of the descriptor algorithm 

        pixel_interval (int, optional): The interval at which keypoints are sampled in the image. If provided,
                                        keypoints are generated at regular intervals over the image based on this value.
                                        If None, the algorithm will automatically determine the keypoints. Defaults to None.

    Returns:
        _type_: Returns a numpy array of descriptors. The type and structure of the array depend on the chosen descriptor algorithm.
                - If "SIFT" is chosen, SIFT descriptors are returned.
                - If "ORB" is chosen, ORB descriptors are returned.
                - If "FAST" is chosen, descriptors are computed using the FAST algorithm for keypoint detection and ORB for descriptor computation.

    Raises:
        AssertionError: If an unsupported descriptor name is provided, the function raises an assertion error.
    """
    if descriptor_name == "SIFT":
        return SIFT_des(img, pixel_interval)
    elif descriptor_name == "ORB":
        return ORB_des(img, pixel_interval)
    elif descriptor_name == "FAST":
        return FAST_des(img, pixel_interval)
    else:
        assert False, f"{descriptor_name} is not supported !!"

# norm scaler

In [7]:
def get_scaled_feature(feature:np.array)->np.array:
    """
    Helper function to scale the provided feature using standard scaling.



    Args:
        feature(np.array): A feature array

    Returns:
        scaled_feature(np.array): The scaled version of the input feature. This is a NumPy array where each
                        feature is scaled to have zero mean and unit variance.
    """
    # Create a StandardScaler object
    scaler = (
        StandardScaler()
    ) 

    # apply scaler transform
    scaled_feature = scaler.fit_transform(feature)
    return scaled_feature

# dimensionality reduction 

In [8]:
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis


def dim_reduction(method: str, train_X: list, train_y: list = None):
    """
    Helper function to perform dimensionality reduction on the input data


    Args:
        method (str): method to be used for dimensionality reduction. Supported values are "PCA" and "LDA".

        train_X (list):  training data features
        train_y (list, optional): training data labels, required for LDA.

    Returns:
        model: The dimensionality reduction model after being fit to the data.
               - If "PCA" is chosen, a PCA model is returned.
               - If "LDA" is chosen, an LDA model is returned.

    Raises:
        AssertionError: If an unsupported method is provided or if train_y is None when using LDA.
    """
    if method == "PCA":
        # Apply PCA for dimensionality reduction
        pca = PCA(n_components=64)
        train_X = pca.fit_transform(train_X)
        return pca

    elif method == "LDA":
        # Apply LDA for dimensionality reduction, requires train_y labels
        if train_y is not None:
            lda = LinearDiscriminantAnalysis(n_components=7)
            train_X = lda.fit_transform(train_X, train_y)
        else:
            assert False, "train_y cannot be None for LDA"
        return lda

    else:
        assert (
            False
        ), f"{method} is not supported as a dimensionality reduction method!!"

# bag of visual words

In [9]:
import cv2
import numpy as np
from tqdm import tqdm
from sklearn.cluster import MiniBatchKMeans


def create_bag_visual_word(
    train_images_filenames,
    descriptor_name,
    pixel_interval,
    k: int,
    apply_Scaling: bool,
):
    """
    Helper function to create a bag of visual words model using the specified feature descriptors from training images.


    Args:
        train_images_filenames: A list of filenames of the training images.

        descriptor_name: name of the descriptor algorithm to be used for feature extraction.

        pixel_interval: pixel interval at which keypoints are sampled in the image for feature extraction.

        k (int): number of clusters to use in k-means. This defines the size of the visual vocabulary.

        apply_Scaling (bool): flag to determine whether to scale the features before clustering.

    Returns:
        codebook: A trained MiniBatchKMeans object that represents the bag of visual words model.
                 
    """
    feature_stack = []

    # Loop through each image, extract features, and optionally scale them
    for file in tqdm(train_images_filenames):
        img = cv2.imread(file)
        feature = get_descriptor_feature(
            img=img, descriptor_name=descriptor_name, pixel_interval=pixel_interval
        )
        if apply_Scaling:
            feature = get_scaled_feature(feature)
        feature_stack.append(feature)

    # Stack all features into a single numpy array
    D = np.vstack(feature_stack)

    # Use MiniBatchKMeans for clustering the features to create the visual words
    codebook = MiniBatchKMeans(
        n_clusters=k,
        verbose=False,
        batch_size=k * 20,
        compute_labels=False,
        reassignment_ratio=10**-4,
        random_state=42,
        n_init="auto",
    )
    codebook.fit(D)

    return codebook

In [10]:
def compute_feature_histogram(features, codebook):
    """
    Calculate the histogram for any feature vector using a pre-trained visual vocabulary (codebook).

    Args:
        features: Calculated keypoints' descriptor (2D array where each row is a descriptor).
        codebook: Pre-trained visual vocabulary (MiniBatchKMeans object).

    Returns:
        Histogram of visual word occurrences.
    """
    words = codebook.predict(features)
    histogram = np.bincount(words, minlength=codebook.n_clusters)
    histogram = histogram.astype(np.float32)
    histogram /= histogram.sum()  # Normalize the histogram
    return histogram

# spatial pyramid technique 

In [11]:
def spatial_pyramid_local_feature_extractor(
    image: np.array,
    descriptor_name: str,
    visual_vocabulary: np.array,
    pixel_interval: int,
    apply_Scaling: bool,
    max_level: int = 2,
):
    """
    WE WILL USE 2 LEVEL
    LEVEL 0: COMPLETE IMAGE
    LEVEL 1: 2X2 GRID
    LEVEL 2: 4X4 GRID
    """
    height, width = image.shape[:2]

    final_feature_vector = []

    for level in range(max_level + 1):
        # NUMBER OF DIVISION
        divisions = 2**level

        region_height = height // divisions
        region_width = width // divisions

        for i in range(divisions):
            for j in range(divisions):
                start_x, start_y = i * region_width, j * region_height
                end_x, end_y = start_x + region_width, start_y + region_height
                region = image[start_y:end_y, start_x:end_x]

                local_features = get_descriptor_feature(
                    img=region,
                    descriptor_name=descriptor_name,
                    pixel_interval=pixel_interval,
                )
                if apply_Scaling:
                    local_features = get_scaled_feature(feature=local_features)
                try:
                    histogram = compute_feature_histogram(
                        local_features, visual_vocabulary
                    )

                    # TODO: LEVEL WISE WEIGHTED HISTOGRAM
                    final_feature_vector.extend(histogram)
                except:
                    continue

    # Normalize final feature vector
    final_feature_vector = np.array(final_feature_vector)
    final_feature_vector = final_feature_vector / np.linalg.norm(final_feature_vector)
    return final_feature_vector

# fisher vector technique 

In [12]:
def compute_fisher_vector(
    img: np.array, descriptor_name: str, pixel_interval: int, num_components: int
):
    descriptors = get_descriptor_feature(
        img=img, descriptor_name=descriptor_name, pixel_interval=pixel_interval
    )
    gmm = GaussianMixture(n_components=num_components, covariance_type="diag")
    gmm.fit(descriptors)
    n_kernels = gmm.n_components
    descriptor_dim = descriptors.shape[1]

    # Initialize Fisher Vector
    fisher_vector = np.zeros(2 * descriptor_dim * n_kernels)

    # Compute posterior probabilities (responsibilities) for each GMM component
    probs = gmm.predict_proba(descriptors)

    # Compute the sufficient statistics
    for i in range(n_kernels):
        # probability for each feature under component i
        prob = probs[:, i].reshape(-1, 1)

        # mean and covariance for component i
        mean_i = gmm.means_[i]
        covar_i = gmm.covariances_[i]  # Fixed line

        # gradients with respect to the means
        grad_mean = (descriptors - mean_i) / covar_i
        fisher_vector[i * descriptor_dim : (i + 1) * descriptor_dim] = np.sum(
            prob * grad_mean, axis=0
        )

        # gradients with respect to the covariances
        grad_covar = (((descriptors - mean_i) ** 2) / covar_i - 1) / np.sqrt(covar_i)
        fisher_vector[
            n_kernels * descriptor_dim
            + i * descriptor_dim : n_kernels * descriptor_dim
            + (i + 1) * descriptor_dim
        ] = np.sum(prob * grad_covar, axis=0)

    #  normalization 
    fisher_vector = np.sign(fisher_vector) * np.sqrt(np.abs(fisher_vector))
    norm = np.sqrt(np.sum(fisher_vector**2))
    if norm > 0:
        fisher_vector /= norm

    return fisher_vector

# classifier models 

In [13]:
def histogram_intersection_kernel(X, Y):
    return np.sum(np.minimum(X, Y), axis=2)

In [14]:
def fit_svm(train_X, train_y, C=1.0, kernel="rbf"):
    if kernel == 'histinter':
        svc = SVC(C=C, kernel=histogram_intersection_kernel)
    else: 
        svc = SVC(C=C, kernel=kernel)
    svc.fit(train_X, train_y)
    return svc


def fit_knn(train_X, train_y, n_neighbors=7, metric="euclidean"):
    knn = KNeighborsClassifier(n_neighbors=n_neighbors, n_jobs=-1, metric=metric)
    knn.fit(train_X, train_y)
    return knn

In [15]:
def get_classifier(train_X, train_y, classifier: str, **kwargs):


    if classifier == "KNN":
        n_neighbors = 7
        if "n_neighbors" in kwargs:
            n_neighbors = kwargs["n_neighbors"]

        metric = "euclidean"
        if "metric" in kwargs:
            metric = kwargs["metric"]

        return fit_knn(
            train_X=train_X, train_y=train_y, n_neighbors=n_neighbors, metric=metric
        )

    if classifier == "SVM":
        C = 1.0
        if "C" in kwargs:
            C = kwargs["C"]

        kernel = "rbf"
        if "kernel" in kwargs:
            kernel = kwargs["kernel"]

        return fit_svm(train_X=train_X, train_y=train_y, C=C, kernel=kernel)

# train and test 

In [16]:
def train_classifier(
    descriptor_name: str,  # Name of the feature descriptor to be used (e.g., SIFT, ORB)
    training_method: str,  # Specifies the training method, e.g., SIMPLE_BOVW, SPATIAL_PYRAMID_BOVW, FISHER_VECTOR
    pixel_interval: int,  # Interval for pixel sampling in image processing
    train_images_filenames:List[str],  # List of filenames (paths) for the training images
    train_labels:List[str],  # Corresponding labels for the training images
    codebook,  # The visual vocabulary (codebook) for feature encoding
    gmm_n_components: int,  # Number of components for the Gaussian Mixture Model (relevant for FISHER_VECTOR method)
    apply_Scaling: bool,
    classifier_model_name: str,
    KNN_classifier_n_neighbor: int = None,
    KNN_classifier_metrics: str = None,
    SVM_classifier_c: int = None,
    SVM_classifier_kernel: str = None,
    dimensionality_reduction_technique: str = None,
):
    print("-" * 10 + "TRAINING FINAL CLASSIFICATION MODEL" + "-" * 10)
    # TRAIN DATA PREP
    train_y = []
    train_X = []

    for file, label in tqdm(
        zip(train_images_filenames, train_labels), total=len(train_images_filenames)
    ):
        img = cv2.imread(file)
        if training_method == "SIMPLE_BOVW":
            dst_features = get_descriptor_feature(
                img=img, descriptor_name=descriptor_name, pixel_interval=pixel_interval
            )

            if apply_Scaling:
                dst_features = get_scaled_feature(feature=dst_features)

            features = compute_feature_histogram(
                features=dst_features, codebook=codebook
            )
        elif training_method == "SPATIAL_PYRAMID_BOVW":
            features = spatial_pyramid_local_feature_extractor(
                image=img,
                descriptor_name=descriptor_name,
                visual_vocabulary=codebook,
                pixel_interval=pixel_interval,
                apply_Scaling=apply_Scaling,
            )
        elif training_method == "FISHER":
            features = compute_fisher_vector(
                img, descriptor_name, pixel_interval, num_components=gmm_n_components
            )

        train_X.append(features)
        train_y.append(label)
    # apply dimensionality reduction
    if dimensionality_reduction_technique:
        dim_reduction_model = dim_reduction(
            method=dimensionality_reduction_technique, train_X=train_X, train_y=train_y
        )
        train_X = dim_reduction_model.transform(train_X)

    else:
        dim_reduction_model = None

    # train classifier
    classifier = get_classifier(
        train_X=train_X,
        train_y=train_y,
        classifier=classifier_model_name,
        n_neighbors=KNN_classifier_n_neighbor,
        metric=KNN_classifier_metrics,
        C=SVM_classifier_c,
        kernel=SVM_classifier_kernel,
    )
    return classifier, dim_reduction_model

In [17]:
def test_classifier(
    descriptor_name: str,
    training_method: str,
    pixel_interval,
    codebook,
    classifier,
    gmm_n_components: int,
    apply_Scaling: bool,
    dim_reduction_model=None,
    test_images_filenames: List[str] = test_images_filenames,
    test_labels: List[str] = test_labels,
):
    print("-" * 10 + "CALCULATING FINAL CLASSIFICATION MODEL ACCURACY" + "-" * 10)
    test_y = []
    test_X = []
    if training_method == "FISHER":
        pass

    for file, label in tqdm(
        zip(test_images_filenames, test_labels), total=len(test_images_filenames)
    ):
        img = cv2.imread(file)

        if training_method == "SIMPLE_BOVW":
            dst_features = get_descriptor_feature(
                img=img, descriptor_name=descriptor_name, pixel_interval=pixel_interval
            )

            features = compute_feature_histogram(
                features=dst_features, codebook=codebook
            )
            if apply_Scaling:
                dst_features = get_scaled_feature(feature=dst_features)

        elif training_method == "SPATIAL_PYRAMID_BOVW":
            features = spatial_pyramid_local_feature_extractor(
                image=img,
                descriptor_name=descriptor_name,
                visual_vocabulary=codebook,
                pixel_interval=pixel_interval,
                apply_Scaling=apply_Scaling,
            )
        elif training_method == "FISHER":
            features = compute_fisher_vector(
                img, descriptor_name, pixel_interval, num_components=gmm_n_components
            )

        test_X.append(features)
        test_y.append(label)
    if dim_reduction_model:
        test_X = dim_reduction_model.transform(test_X)

    pred_classes = classifier.predict(test_X)
    # accuracy
    accuracy = accuracy_score(test_y, pred_classes)
    return accuracy

# optuna hyper parameter tuning 

In [18]:
available_descriptors = [
    "SIFT",
    "ORB",
    "FAST",
]
available_training_method = ["SIMPLE_BOVW", "SPATIAL_PYRAMID_BOVW", "FISHER"]

# TODO: implement t-SNE
available_dimensionality_reduction_technique = ["PCA", "LDA", None]
available_classifier_models = [
    "KNN",
    "SVM",
]

available_KNN_classifier_metrics = [
    "euclidean",  # Euclidean distance
    "minkowski",  # Minkowski distance, general form of Euclidean and Manhattan
    "chebyshev",  # Chebyshev distance
    "hamming",  # Hamming distance, for categorical variables
    "cosine",  # Cosine similarity
    "jaccard",  # Jaccard similarity
]

available_SVM_classifier_kernels = [
    "linear",  # Linear kernel
    "poly",  # Polynomial kernel
    "rbf",  # Radial Basis Function (Gaussian) kernel
    "sigmoid",  # Sigmoid kernel
    "histinter", # Histogram intersection kernel
]
available_use_of_norm_scaler = ["True", "False"]

In [19]:
def objective(trail):
    wandb.init(project="FINDING_BEST_COMBINATION_GUNJAN")

    descriptor_name = trail.suggest_categorical(
        "descriptor_name", available_descriptors
    )

    pixel_interval = trail.suggest_int("pixel_interval", 5, 30)

    training_method = trail.suggest_categorical(
        "training_method", available_training_method
    )

    selected_classifier_model = trail.suggest_categorical(
        "classifier_model", available_classifier_models
    )
    if selected_classifier_model == "KNN":
        selected_KNN_classifier_n_neighbor = trail.suggest_int(
            "KNN_classifier_n_neighbor", 5, 100
        )
        selected_KNN_classifier_metrics = trail.suggest_categorical(
            "KNN_classifier_metrics", available_KNN_classifier_metrics
        )

        selected_SVM_classifier_c = None
        selected_SVM_classifier_kernels = None

    elif selected_classifier_model == "SVM":
        selected_SVM_classifier_c = trail.suggest_int("SVM_classifier_c", 1, 10)
        selected_SVM_classifier_kernels = trail.suggest_categorical(
            "SVM_classifier_kernels", available_SVM_classifier_kernels
        )

        selected_KNN_classifier_n_neighbor = None
        selected_KNN_classifier_metrics = None

    apply_Scaling = trail.suggest_categorical(
        "use_of_norm_scaler", available_use_of_norm_scaler
    )
    apply_Scaling = eval(apply_Scaling)

    selected_dimensionality_reduction_technique = trail.suggest_categorical(
        "dimensionality_reduction_technique",
        available_dimensionality_reduction_technique,
    )
    try:
        if training_method in ["SIMPLE_BOVW", "SPATIAL_PYRAMID_BOVW"]:
            bovw_num_cluster = trail.suggest_int("bovw_num_cluster", 50, 1000)
            codebook = create_bag_visual_word(
                descriptor_name=descriptor_name,
                k=bovw_num_cluster,
                train_images_filenames=train_images_filenames,
                pixel_interval=pixel_interval,
                apply_Scaling=apply_Scaling,
                dimensionality_reduction_technique=selected_dimensionality_reduction_technique,
            )
            gmm_n_components = None
        elif training_method == "FISHER":
            gmm_n_components = trail.suggest_int("gmm_n_components", 10, 100)
            codebook = None
            bovw_num_cluster = None

        classifier, dim_reduction_model = train_classifier(
            descriptor_name=descriptor_name,
            train_images_filenames=train_images_filenames,
            training_method=training_method,
            train_labels=train_labels,
            codebook=codebook,
            pixel_interval=pixel_interval,
            gmm_n_components=gmm_n_components,
            apply_Scaling=apply_Scaling,
            classifier_model_name=selected_classifier_model,
            KNN_classifier_n_neighbor=selected_KNN_classifier_n_neighbor,
            KNN_classifier_metrics=selected_KNN_classifier_metrics,
            SVM_classifier_c=selected_SVM_classifier_c,
            SVM_classifier_kernel=selected_SVM_classifier_kernels,
            dimensionality_reduction_technique=selected_dimensionality_reduction_technique,
        )
        accuracy = test_classifier(
            descriptor_name=descriptor_name,
            pixel_interval=pixel_interval,
            training_method=training_method,
            codebook=codebook,
            classifier=classifier,
            gmm_n_components=gmm_n_components,
            apply_Scaling=apply_Scaling,
            dim_reduction_model=dim_reduction_model,
        )

    except Exception as error:
        print(">" * 10 + "ERROR" + "<" * 10)
        print(str(error))
        print(">" * 10 + "<" * 10)
        return 0

    wandb.log(
        {
            "bovw_num_cluster": bovw_num_cluster,
            "pixel_interval": pixel_interval,
            "gmm_n_components": gmm_n_components,
            "accuracy": accuracy,
        }
    )
    return accuracy

In [20]:
# if __name__ == "__main__":
#     study = optuna.create_study(
#         direction="maximize",
#         storage="sqlite:///db.sqlite3",
#         pruner=optuna.pruners.SuccessiveHalvingPruner(),
#     )
#     study.optimize(
#         objective,
#         n_trials=1000000000000000000,
#         timeout=600000000000000000,
#         callbacks=[wandbc], # weight and bias connection
#     )

# cross validation with best parameters 

In [21]:
# BEST HYPER PARAMETERS
bovw_num_cluster = 340
descriptor_name = "SIFT"
training_method = "SIMPLE_BOVW"
pixel_interval = 5
apply_Scaling = False
classifier_model_name = "SVM"
SVM_classifier_c = 1
SVM_classifier_kernel = "sigmoid"
dimensionality_reduction_technique = "LDA"

In [22]:
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [23]:
performance_metrics = []
bovw_num_cluster = bovw_num_cluster
codebook = create_bag_visual_word(
    descriptor_name=descriptor_name,
    k=bovw_num_cluster,
    train_images_filenames=train_images_filenames,
    pixel_interval=pixel_interval,
    apply_Scaling=apply_Scaling
)
for train_index, val_index in kfold.split(train_images_filenames):
    # Split the data into training and validation sets
    X_train_files, X_val_files = [train_images_filenames[i] for i in train_index], [
        train_images_filenames[i] for i in val_index
    ]
    y_train_labels, y_val_labels = [train_labels[i] for i in train_index], [
        train_labels[i] for i in val_index
    ]

   

    classifier, dim_reduction_model = train_classifier(
        descriptor_name=descriptor_name,
        train_images_filenames=X_train_files,
        training_method=training_method,
        train_labels=y_train_labels,
        codebook=codebook,
        pixel_interval=pixel_interval,
        gmm_n_components=None,
        apply_Scaling=apply_Scaling,
        classifier_model_name=classifier_model_name,
        # KNN_classifier_n_neighbor=selected_KNN_classifier_n_neighbor,
        # KNN_classifier_metrics=selected_KNN_classifier_metrics,
        SVM_classifier_c=SVM_classifier_c,
        SVM_classifier_kernel=SVM_classifier_kernel,
        dimensionality_reduction_technique=dimensionality_reduction_technique,
    )
    accuracy = test_classifier(
        descriptor_name=descriptor_name,
        pixel_interval=pixel_interval,
        training_method=training_method,
        codebook=codebook,
        classifier=classifier,
        gmm_n_components=None,
        apply_Scaling=apply_Scaling,
        dim_reduction_model=dim_reduction_model,
        test_images_filenames=X_val_files,
        test_labels=y_val_labels,
    )
    print("#" * 100)
    print(f"accuracy: {accuracy}")
    print("#" * 100)
    performance_metrics.append(accuracy)

100%|██████████| 1881/1881 [00:39<00:00, 47.76it/s]


----------TRAINING FINAL CLASSIFICATION MODEL----------


100%|██████████| 1504/1504 [00:44<00:00, 33.93it/s]


----------CALCULATING FINAL CLASSIFICATION MODEL ACCURACY----------


100%|██████████| 377/377 [00:11<00:00, 32.23it/s]


...

####################################################################################################
accuracy: 0.8297872340425532
####################################################################################################
----------TRAINING FINAL CLASSIFICATION MODEL----------


100%|██████████| 1505/1505 [00:45<00:00, 32.93it/s]


----------CALCULATING FINAL CLASSIFICATION MODEL ACCURACY----------


100%|██████████| 376/376 [00:11<00:00, 33.06it/s]

####################################################################################################
accuracy: 0.8218085106382979
####################################################################################################


In [24]:
# Average the results
average_performance = sum(performance_metrics) / len(performance_metrics)
print(f"Average Performance: {average_performance}")

Average Performance: 0.8293343303798182
